<a href="https://colab.research.google.com/github/naderessam10/BlockChain/blob/main/BlockChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Python program to create Blockchain

# For timestamp
import datetime

# Calculating the hash
# in order to add digital
# fingerprints to the blocks
import hashlib

import timeit
# To store data
# in our blockchain
import json


class Blockchain:

	# This function is created
	# to create the very first
	# block and set it's hash to "0"
	def __init__(self,proof=1,previous_hash='0',index=0):
		self.index=index
		self.chain = []
		self.FirstBlock=self.create_block(proof=proof, previous_hash=previous_hash)

	# This function is created
	# to add further blocks
	# into the chain
	def create_block(self, proof, previous_hash):
		block = {'index': self.index,
				'timestamp': str(datetime.datetime.now()),
				'proof': proof,
				'previous_hash': previous_hash}
		self.index +=1
		self.chain.append(block)
		return block
	
	# This function is created
	# to display the previous block
	def print_previous_block(self):
		return self.chain[-1]
	def print_attacked_block(self):
		return self.chain[-2]
	# This is the function for proof of work
	# and used to successfully mine the block
	def proof_of_work(self, previous_proof):
		new_proof = 1
		check_proof = False
		
		while check_proof is False:
			hash_operation = hashlib.sha256(
				str(new_proof**2 - previous_proof**2).encode()).hexdigest()
			if hash_operation[:5] == '00000':
				check_proof = True
			else:
				new_proof += 1
				
		return new_proof

	def hash(self, block):
		encoded_block = json.dumps(block, sort_keys=True).encode()
		return hashlib.sha256(encoded_block).hexdigest()

	def chain_valid(self, chain):
		previous_block = chain[0]
		block_index = 1
		
		while block_index < len(chain):
			block = chain[block_index]
			if block['previous_hash'] != self.hash(previous_block):
				return False
			
			previous_proof = previous_block['proof']
			proof = block['proof']
			hash_operation = hashlib.sha256(
				str(proof**2 - previous_proof**2).encode()).hexdigest()
			
			if hash_operation[:5] != '00000':
				return False
			previous_block = block
			block_index += 1
		
		return True



# Create the object
# of the class blockchain
blockchain = Blockchain()

# Mining a new block
def mine_block(chain):
	start = timeit.default_timer()
	previous_block = chain.print_previous_block()
	previous_proof = previous_block['proof']
	proof = chain.proof_of_work(previous_proof)
	previous_hash = chain.hash(previous_block)
	block = chain.create_block(proof, previous_hash)
	
	response = {'message': 'A block is MINED',
				'index': block['index'],
				'timestamp': block['timestamp'],
				'proof': block['proof'],
				'previous_hash': block['previous_hash']}
	stop = timeit.default_timer()
	print('Time: ', stop - start)
	print(response)
def mine_in_attack(power,length):
	#first block for attacker
	attacked_block = blockchain.print_attacked_block()
	
	attacked_proof = attacked_block['proof']
	proof_attack = blockchain.proof_of_work(attacked_proof)
	attacked_hash = blockchain.hash(attacked_block)
	attacked_index=attacked_block['index']+1
	attackedchain = Blockchain(proof_attack,attacked_hash,attacked_index)
	firstAttackedBlock=attackedchain.FirstBlock
	responseAttack = {'message': 'A block is MINED',
				'index': firstAttackedBlock['index'],
				'timestamp': firstAttackedBlock['timestamp'],
				'proof': firstAttackedBlock['proof'],
				'previous_hash': firstAttackedBlock['previous_hash']}
	#first block for user
	previous_block = blockchain.print_previous_block()
	previous_proof = previous_block['proof']
	proof = blockchain.proof_of_work(previous_proof)
	previous_hash = blockchain.hash(previous_block)
	previous_index=previous_block['index']+1
	userchain = Blockchain(proof,previous_hash,previous_index)
	firstUserBlock=userchain.FirstBlock
	responseUser = {'message': 'A block is MINED',
				'index': firstUserBlock['index'],
				'timestamp': firstUserBlock['timestamp'],
				'proof': firstUserBlock['proof'],
				'previous_hash': firstUserBlock['previous_hash']}
	
	userCounter=0
	attackerCounter=0
	userTurn=True
	indexattacker=round(power*length)-1
	length-=2
  
	for i in range(length):
		if i<indexattacker:
			if i==0:
				print('Start attacker turn:')
				print(responseAttack)

			mine_block(attackedchain)
			attackerCounter+=1
		else:
			if userTurn:
				print('Start user turn:')
				print(responseUser)
				userTurn=False
			mine_block(userchain)
			userCounter+=1

	if userCounter>attackerCounter:
		blockchain.chain.extend(userchain.chain)
		return False
	else:
		blockchain.chain.pop()
		blockchain.chain.extend(attackedchain.chain)
		return True
# Display blockchain in json format
def display_chain():
	response = {'chain': blockchain.chain,
				'length': len(blockchain.chain)}
	print(response)

# Check validity of blockchain
def valid():
	valid = blockchain.chain_valid(blockchain.chain)
	
	if valid:
		response = {'message': 'The Blockchain is valid.'}
	else:
		response = {'message': 'The Blockchain is not valid.'}
	print(response)

def main():
	numberOfFirstBlock=6
	power=0.4
	numOfRound=10
	print(f'First mine normal {numberOfFirstBlock} blocks:')
	for i in range(numberOfFirstBlock):
		mine_block(blockchain)

	print(f'Starting attack on node {numberOfFirstBlock}:')
	print(f'Attacker has {power} of total computational power:')
	if mine_in_attack(power,numOfRound):
		print('Succesed attack')
		print(f'Attacker mine {round(power*numOfRound)} blocks')
		print(f'User mine {round((1-power)*numOfRound)}')
		print(f'The total blocks in the chain after popping last block and adding the attacker blocks {len(blockchain.chain)}')
	else:
		print ('faild attack')
		print(f'Attacker mine {round(power*numOfRound)} blocks')
		print(f'User mine {round((1-power)*numOfRound)}')
		print(f'The total blocks in the chain after adding the user blocks {len(blockchain.chain)}')
	display_chain()
  
if __name__ == "__main__":
    main()

First mine normal 6 blocks:
Time:  1.2437283780000143
{'message': 'A block is MINED', 'index': 1, 'timestamp': '2022-02-10 17:36:01.393147', 'proof': 632238, 'previous_hash': 'b9ba4ff2bf17336aa073cccff0c5703c0151f66708075d61cc52563c805045c2'}
Time:  0.805993691000026
{'message': 'A block is MINED', 'index': 2, 'timestamp': '2022-02-10 17:36:02.200265', 'proof': 403091, 'previous_hash': '503c0e269330b9997100d92f1e124de2c95d31cf87df66c247d3f4a7c10f913d'}
Time:  1.4115721239999743
{'message': 'A block is MINED', 'index': 3, 'timestamp': '2022-02-10 17:36:03.613139', 'proof': 714736, 'previous_hash': '29d6c17ebb0f85a3fa116c543a5ba5a9141b501aabb144c645c14384c3cffa38'}
Time:  0.9344876249999743
{'message': 'A block is MINED', 'index': 4, 'timestamp': '2022-02-10 17:36:04.549354', 'proof': 476581, 'previous_hash': '5f4200d8b36315a159bb7880d760b55026ff24db246756c9d42c90671a78d7ce'}
Time:  0.84985190499998
{'message': 'A block is MINED', 'index': 5, 'timestamp': '2022-02-10 17:36:05.400364', 'p